In [ ]:
pip install gymnasium stable-baselines3 torch gradio


In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np

class WebAutomationEnv(gym.Env):
    def __init__(self):
        super(WebAutomationEnv, self).__init__()
        self.state = 0
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Discrete(10)

    def step(self, action):
        reward = 0
        if action == 0:
            reward = 1
        elif action == 1:
            reward = 0.5
        elif action == 2:
            reward = 2

        self.state = (self.state + 1) % 10
        done = self.state == 9
        return self.state, reward, done, False, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.state = 0
        return self.state, {}


In [ ]:
env = WebAutomationEnv()
model = PPO("MlpPolicy", env, verbose=1, device="cuda")
model.learn(total_timesteps=5000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9        |
|    ep_rew_mean     | 11       |
| time/              |          |
|    fps             | 676      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9          |
|    ep_rew_mean          | 11.3       |
| time/                   |            |
|    fps                  | 529        |
|    iterations           | 2          |
|    time_elapsed         | 7          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02671265 |
|    clip_fraction        | 0.275      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.0

In [ ]:
import gradio as gr

def ai_decision(state):
    action, _ = model.predict(state)
    return f"AI decides to: {['Click', 'Scroll', 'Type Input'][action]}"

gr.Interface(fn=ai_decision, inputs="number", outputs="text").launch(share=True)
